In [1]:
from typing import List, Optional, Callable


class Block:
    def __init__(self):
        self.parent: Optional["Container"] = None
        self.has_children: bool = False


class Container(Block):
    def __init__(self):
        super().__init__()
        self.has_children = True
        self.children: List[Block] = []

    def append(self, block: Block):
        self.children.append(block)
        block.parent = self


class Row(Container): ...


class Column(Container): ...


class Text(Block):
    def __init__(
        self,
        text: str = "",
        hue: Optional[int] = None,
        *args,
        **kwargs,
    ):
        super().__init__()
        self.text = text
        self.hue = hue


class Button(Block):
    def __init__(
        self,
        label: str = "",
        hue: Optional[int] = None,
        *args,
        **kwargs,
    ):
        super().__init__()
        self.label = label
        self.hue = hue
        self.click_handler: Optional[Callable] = None
        self.click_args: Optional[List[Block]] = None

    def click(self, handler: Callable, args: List[Block]):
        self.click_handler = handler
        self.click_args = args


class Html(Block):
    def __init__(
        self,
        text: str = "",
        color: Optional[int] = None,
        centered: bool = False,
        background: bool = False,
        scrollbar: bool = False,
        *args,
        **kwargs,
    ):
        super().__init__()
        self.text = text
        self.color = color
        self.centered = centered
        self.background = background
        self.scrollbar = scrollbar


class TextEntry(Block): ...


class GumpBuilder:
    class Scope:
        def __init__(self, builder: "GumpBuilder", block: Container):
            self.builder = builder
            self.block = block

        def __enter__(self):
            self.builder.current.append(self.block)
            self.builder.current = self.block
            return self.block

        def __exit__(self, exc_type, exc_val, exc_tb):
            self.builder.current = self.block.parent if self.block.parent else self.builder.root

    def __init__(self):
        self.root: Container = Row()
        self.current: Container = self.root

    def Row(self):
        return self.Scope(self, Row())

    def Column(self):
        return self.Scope(self, Column())

    def Text(
        self,
        text: str = "",
        hue: Optional[int] = None,
        *args,
        **kwargs,
    ):
        block = Text(text, hue, *args, **kwargs)
        self.current.append(block)
        return block

    def Button(
        self,
        label: str = "",
        hue: Optional[int] = None,
        *args,
        **kwargs,
    ):
        block = Button(label, hue, *args, **kwargs)
        self.current.append(block)
        return block

    def Html(
        self,
        text: str = "",
        color: Optional[int] = None,
        centered: bool = False,
        background: bool = False,
        scrollbar: bool = False,
        *args,
        **kwargs,
    ):
        block = Html(text, color, centered, background, scrollbar, *args, **kwargs)
        self.current.append(block)
        return block

    def TextEntry(
        self,
        *args,
        **kwargs,
    ):
        block = TextEntry(*args, **kwargs)
        self.current.append(block)
        return block

In [2]:
gr = GumpBuilder()
with gr.Row():
    with gr.Column():
        pass
    with gr.Column():
        with gr.Row():
            gr.Text("Hello, World!", hue=1153)
        with gr.Row():
            gr.Text("This is a test.", hue=1152)